# **NYU Wagner - Python Coding for Public Policy**
# Class 5: APIs

## Ways to get data

Method | How it happens | Pros | Cons
--- | :--- | :--- | :---
**Bulk** | Download, someone hands you a flash drive, etc. | Fast, one-time transfer | Can be large
**Scraping** | Data only available through a web site, PDF, or doc | You can turn anything into data | Tedious; fragile
**APIs** | If organization makes one available | Usually allows some filtering; can always pull latest-and-greatest | Requires network connection for every pull; higher barrier to entry (reading documentation); subject to availability and performance of API

### Scraping

Common tools:

- [Beautiful Soup package](https://realpython.com/beautiful-soup-web-scraper-python/)
- [pandas' `read_html()`](https://pandas.pydata.org/docs/user_guide/io.html#html)

In [1]:
import pandas as pd

tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population', match='Rank')
pop = tables[0]
pop

,Rank,Country / Dependency,Region,Population,Percentage of the world,Date,Source (official or from the United Nations),Notes,Unnamed: 8
0,–,World,NaN,7938495000,100%,24 Mar 2022,UN projection[1],NaN,NaN
1,1,China,Asia,1412600000,NaN,31 Dec 2021,National annual estimate[2],The population figure refers to mainland China...,NaN
2,2,India,Asia,1374543575,NaN,24 Mar 2022,National population clock[3],The figure includes the population of Jammu an...,NaN
3,3,United States,Americas,332580125,NaN,24 Mar 2022,National population clock[4],The figure includes the 50 states and the Dist...,NaN
4,4,Indonesia,Asia[b],272248500,NaN,1 Jul 2021,National annual estimate[5],NaN,NaN
...,...,...,...,...,...,...,...,...,...
237,–,Niue (New Zealand),Oceania,1549,NaN,1 Jul 2021,National annual projection[92],NaN,NaN
238,–,Tokelau (New Zealand),Oceania,1501,NaN,1 Jul 2021,National annual projection[92],NaN,NaN
239,195,Vatican City,Europe,825,NaN,1 Feb 2019,Monthly national estimate[196],The total population of 825 consisted of 453 r...,NaN
240,–,Cocos (Keeling) Islands (Australia),Oceania,573,NaN,30 Jun 2020,National annual estimate[195],NaN,NaN


## Data is only available if it's available

## API calls in the wild

1. Go to [Candidates page on fec.gov](https://www.fec.gov/data/candidates/?has_raised_funds=true&is_active_candidate=true).
1. Right click and `Inspect`.
   - [More info about opening Developer Tools in various browsers.](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_are_browser_developer_tools#how_to_open_the_devtools_in_your_browser)
1. Go to the `Network` tab and reload.
1. Filter to `XHR`.
1. Click the API call.

### Parts of a URL

![URL structure](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL/mdn-url-all.png)

[source](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL#basics_anatomy_of_a_url)

For APIs:

- Often split into "base URL" + "endpoint"
- Anchors aren't relevant

### API documentation

[FEC API](https://api.open.fec.gov/developers/)

### Try it out

1. Visit https://www.fec.gov/data/candidates/
1. In the Network tab's request list, right-click the API call.
1. Click `Open in New Tab`.
1. Replace the API key with `DEMO_KEY`.

## API calls from Python

Usually one of two ways:

- A software development kit (SDK)
   - Only if the API provider offers one
   - Abstracts the details away
   - May have limitations
- [The `requests` package](https://docs.python-requests.org/)

In [2]:
import requests

params = {
    'api_key': 'DEMO_KEY',
    'q': 'Jimmy McMillan',
    'sort': '-first_file_date'
}
response = requests.get('https://api.open.fec.gov/v1/candidates/', params=params)
data = response.json()
data

{'api_version': '1.0',
 'results': [{'candidate_id': 'P60016805',
   'election_years': [2016],
   'last_file_date': '2015-10-13',
   'name': 'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH',
   'incumbent_challenge_full': 'Open seat',
   'load_date': '2018-02-17T09:16:20+00:00',
   'office_full': 'President',
   'last_f2_date': '2015-10-13',
   'first_file_date': '2015-10-13',
   'candidate_status': 'N',
   'election_districts': ['00'],
   'district_number': 0,
   'state': 'US',
   'party': 'REP',
   'cycles': [2016, 2018],
   'party_full': 'REPUBLICAN PARTY',
   'district': '00',
   'incumbent_challenge': 'O',
   'active_through': 2016,
   'has_raised_funds': False,
   'inactive_election_years': None,
   'federal_funds_flag': False,
   'candidate_inactive': False,
   'flags': 'P60016805',
   'office': 'P'},
  {'candidate_id': 'P60003290',
   'election_years': [1996, 2012],
   'last_file_date': '2011-02-07',
   'name': 'MCMILLAN, JIMMY (AKA) JAMES ',
   'incumbent_challenge_full': 'Challenger'

### Retrieving nested data

In [3]:
data['results'][0]['name']

'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH'

### Reading into a DataFrame

In [4]:
params = {'api_key': 'DEMO_KEY'}
response = requests.get('https://api.open.fec.gov/v1/candidates/', params=params)
data = response.json()

pd.DataFrame(data['results'])

,incumbent_challenge_full,candidate_inactive,has_raised_funds,active_through,last_file_date,flags,candidate_id,state,party_full,party,...,inactive_election_years,district,last_f2_date,federal_funds_flag,election_years,office,election_districts,cycles,first_file_date,name
0,None,False,False,2024,2021-05-05,P40006033,P40006033,US,PEACE AND FREEDOM,PAF,...,None,00,2021-04-28,False,[2024],P,[00],[2022],2021-04-28,"12-INCH COCK PLEASE, CAN YOU SUCK MY"
1,Challenger,False,False,2020,2019-04-23,P00011569,P00011569,US,NONE,NNE,...,None,00,2019-04-23,False,[2020],P,[00],"[2020, 2022]",2019-04-23,"753, JO"
2,Challenger,False,False,2004,2002-01-30,P40002172,P40002172,US,INDEPENDENT,IND,...,None,00,2002-01-30,False,[2004],P,[00],"[2002, 2004]",2002-01-30,"AABBATTE, MICHAEL THOMAS WITORT"
3,Challenger,False,False,2022,2021-06-04,S2CO00175,S2CO00175,CO,REPUBLICAN PARTY,REP,...,None,00,2021-06-04,False,[2022],S,[00],[2022],2021-06-04,"AADLAND, ERIK"
4,Open seat,False,True,2022,2021-12-27,H2CO07170,H2CO07170,CO,REPUBLICAN PARTY,REP,...,None,07,2021-12-27,False,[2022],H,[07],[2022],2021-12-27,"AADLAND, ERIK"
5,Challenger,False,True,2020,2020-03-24,H0UT03227,H0UT03227,UT,REPUBLICAN PARTY,REP,...,None,03,2020-03-24,False,[2020],H,[03],[2020],2020-03-24,"AADLER, TIM"
6,Challenger,False,False,2022,None,H4UT04052,H4UT04052,UT,REPUBLICAN PARTY,REP,...,None,03,None,False,"[2014, 2022]",H,"[04, 03]","[2014, 2022]",None,"AALDERS, TIM"
7,Challenger,False,False,2022,2022-03-21,H2UT03280,H2UT03280,UT,REPUBLICAN PARTY,REP,...,None,03,2022-03-21,False,[2022],H,[03],[2022],2022-03-21,"AALDERS, TIM"
8,Open seat,False,True,2018,2018-04-23,S2UT00229,S2UT00229,UT,CONSTITUTION PARTY,CON,...,None,00,2018-04-23,False,"[2012, 2018]",S,"[00, 00]","[2012, 2014, 2016, 2018, 2020]",2012-02-08,"AALDERS, TIMOTHY NOEL"
9,Open seat,False,True,2020,2019-10-17,H0TX22260,H0TX22260,TX,REPUBLICAN PARTY,REP,...,None,22,2019-10-17,False,[2020],H,[22],[2020],2019-10-17,"AALOORI, BANGAR REDDY"


## Back to 311 data

From [NYC Open Data Portal dataset page](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/data), click `Export` -> `SODA API` -> `API Docs`.

### Most open data sites have APIs

Often built on platforms that provide them, e.g.

- [NYC Open Data Portal](https://opendata.cityofnewyork.us/) built on [Socrata](https://dev.socrata.com/)
- [data.gov built on CKAN](https://www.data.gov/developers/apis)

### Example: 311 requests from the last week

In [5]:
from datetime import datetime, timedelta

now = datetime.utcnow()
now

datetime.datetime(2022, 3, 24, 15, 5, 57, 888433)

In [6]:
start = now - timedelta(weeks=1)
start

datetime.datetime(2022, 3, 17, 15, 5, 57, 888433)

In [7]:
start.isoformat()

'2022-03-17T15:05:57.888433'

Using the [Socrata query language (SoQL)](https://dev.socrata.com/docs/queries/):

In [8]:
data_id = 'erm2-nwe9'
params = {
    '$where': f"created_date between '{start.isoformat()}' and '{now.isoformat()}'"
}

url = f'https://data.cityofnewyork.us/resource/{data_id}.json'
response = requests.get(url, params=params)
data = response.json()

data

[{'unique_key': '53672466',
  'created_date': '2022-03-17T15:06:07.000',
  'closed_date': '2022-03-18T10:15:00.000',
  'agency': 'DOT',
  'agency_name': 'Department of Transportation',
  'complaint_type': 'Street Condition',
  'descriptor': 'Pothole',
  'incident_zip': '11356',
  'incident_address': '20-38 127 STREET',
  'street_name': '127 STREET',
  'cross_street_1': '20 AVENUE',
  'cross_street_2': '22 AVENUE',
  'address_type': 'ADDRESS',
  'city': 'COLLEGE POINT',
  'facility_type': 'N/A',
  'status': 'Closed',
  'resolution_description': 'The Department of Transportation inspected this complaint and repaired the problem.',
  'resolution_action_updated_date': '2022-03-18T10:15:00.000',
  'community_board': '07 QUEENS',
  'bbl': '4041710035',
  'borough': 'QUEENS',
  'x_coordinate_state_plane': '1028163',
  'y_coordinate_state_plane': '224011',
  'open_data_channel_type': 'UNKNOWN',
  'park_facility_name': 'Unspecified',
  'park_borough': 'QUEENS',
  'latitude': '40.78142559574469'

In [9]:
pd.DataFrame(data)

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,...,intersection_street_1,intersection_street_2,landmark,bridge_highway_name,bridge_highway_segment,taxi_pick_up_location,bridge_highway_direction,road_ramp,due_date,taxi_company_borough
0,53672466,2022-03-17T15:06:07.000,2022-03-18T10:15:00.000,DOT,Department of Transportation,Street Condition,Pothole,11356,20-38 127 STREET,127 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,53672195,2022-03-17T15:06:17.000,2022-03-17T16:26:15.000,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,11206,556 FLUSHING AVENUE,FLUSHING AVENUE,...,LEE AVENUE,MARCY AVENUE,FLUSHING AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,53671712,2022-03-17T15:06:20.000,2022-03-17T15:12:33.000,NYPD,New York City Police Department,Illegal Parking,Blocked Crosswalk,10019,200 CPS,CPS,...,7 AVENUE,COLUMBUS CIRCLE,CENTRAL PARK SOUTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,53675802,2022-03-17T15:06:29.000,NaN,DHS,Department of Homeless Services,Homeless Person Assistance,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,F,Mezzanine,NaN,NaN,NaN,NaN,NaN
4,53676579,2022-03-17T15:07:00.000,2022-03-17T15:07:00.000,DEP,Department of Environmental Protection,Water System,No Water (WNW),11211,59 LEE AVENUE,LEE AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,53673788,2022-03-17T18:01:07.000,NaN,DSNY,Department of Sanitation,Dead Animal,Raccoon,11214,18 AVENUE,18 AVENUE,...,18 AVENUE,86 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,53673569,2022-03-17T18:01:12.000,NaN,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,11237,20 MORGAN AVENUE,MORGAN AVENUE,...,ROCK STREET,THAMES STREET,MORGAN AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,53677865,2022-03-17T18:01:39.000,2022-03-19T06:22:04.000,DSNY,Department of Sanitation,Dirty Condition,Trash,10309,6264 AMBOY ROAD,AMBOY ROAD,...,SHARROTT AVENUE,BLOOMINGDALE ROAD,AMBOY ROAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,53676114,2022-03-17T18:02:05.000,2022-03-17T19:37:20.000,NYPD,New York City Police Department,Blocked Driveway,No Access,11372,37-29 93 STREET,93 STREET,...,ELMHURST AVENUE,FLUSHING LINE,93 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Coincidence there were exactly 1,000 results?

## Pagination

- Most APIs limit the number of results returned.
- [Socrata defaults to 1,000.](https://dev.socrata.com/docs/queries/limit.html)
- Need to use a loop with parameters like [`$limit`](https://dev.socrata.com/docs/queries/limit.html)+[`$offset`](https://dev.socrata.com/docs/queries/offset.html) (Socrata) or `page`+`per_page` ([FEC](https://api.open.fec.gov/developers/))
   - [`append()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html) to DataFrame

## Things are going to differ by API

- Endpoints
- Supported parameters
- Response structure
   - [`json_normalize()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html) can help
- Quality of documentation
- Helpfulness of errors
- Size/helpfulness of community

Gotta read and experiment.

## [Homework 5](https://padmgp-4506-spring.rcnyu.org/user-redirect/notebooks/class_materials/hw_5.ipynb)

## Homework 6

- a.k.a. the final project
- Open-ended
   - Be creative, and just the right amount of ambitious
- **Goal:** Prove or disprove a hypothesis using skills learned in this class

In real/ideal world, start with specific question and find data to answer it:

![project flow](https://textbook.coleridgeinitiative.org/ChapterIntro/figures/projectflow.png)

_Source: [Big Data and Social Science](https://textbook.coleridgeinitiative.org/chap-intro.html#the-structure-of-the-book)_

Data needed often doesn't exist or is hard (or impossible) to find/access

![project flow](https://textbook.coleridgeinitiative.org/ChapterIntro/figures/projectflow.png)

Safer approach, for the purposes of this assignment:

1. Find dataset that seems interesting
  - Use at least one dataset that you aren't familiar with, from this class or elsewhere.
  - [NYC OpenData](https://opendata.cityofnewyork.us/), [data.gov](https://www.data.gov/), and [Kaggle](https://www.kaggle.com/datasets) have many many options.
  - Finding a dataset available in CSV or JSON is recommended, though [pandas can read other formats](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).
1. Inspect the data a bit.
1. Come up with a question that the data is capable of answering and _isn't trivial to answer_.
  - If you aren't sure, ask.
1. Come up with a hypothesis (a.k.a. a guess of the answer to the question).
1. Submit the proposal.

### Proposal

Post responses to the following as a new Conversation under [the `HW6 proposals` Discussion](https://brightspace.nyu.edu/d2l/le/156784/discussions/topics/281277/View) by next class:

- **What dataset are you going to use?**
   - Please include a link. If multiple, how are you going to merge/join them?
- **What's the question you are trying to answer?**
   - It should be specific, and objectively answerable through the data available.
- **What's your hypothesis?**

- Your question/hypothesis doesn't need to be something novel; confirming something you read in the news is fine.
- You won't be graded on the scientific soundness of your work.
   - Important to think through and note assumptions/caveats of your approach
- The sooner you post your proposal, the sooner you'll get feedback.

#### Simplified example

- **Dataset:** [Recycling Diversion and Capture Rates](https://data.cityofnewyork.us/Environment/Recycling-Diversion-and-Capture-Rates/gaq9-z3hz)
- **Question:** From 2016 to 2019, what community district increased their diversion (recycling) rate the most?
- **Hypothesis:** [Bushwick](https://communityprofiles.planning.nyc.gov/brooklyn/4), because it's gentrified over that time, and hipsters love to recycle.

### Once you start

- Create a new notebook to do the actual analysis; you will turn that in separately.
- Go back and find any information that's available _around_ the data, to get a better understanding of what it contains and means.
  - Might include a data dictionary
  - Might involve poking around a government agency's web site to understand their processes
  - Understand what all the different columns and values represent

### Analysis requirements

_on top of [general assignment requirements](https://github.com/afeld/python-public-policy/blob/main/syllabus.md#assignments)_

<!-- make sure edits here are reflected in extras/scripts/hw_6_check.py -->

- **Read like a blog post**
    - Pretend you're explaining to a Wagner student who hasn't taken this class. You don't need to teach them Python, but they should be able to follow what's going on.
    - Re-state the question, hypothesis, and data source(s) with link(s)
    - Walk the reader through what you're doing in every step and what they should be taking away from it.
    - [Markdown](https://www.markdownguide.org/basic-syntax/) can be used in text cells for formatting.
    - Include any dead ends you hit.
    - Have a conclusion that speaks to your question and hypothesis.
- **Use pandas**
- **Not be trivial**, requiring:
    - At least 40 lines of code to come to a conclusion
    - Transforming data through [grouping](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html), [merging](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging), and/or [reshaping](https://pandas.pydata.org/docs/user_guide/reshaping.html) of DataFrames
    - Operations that aren't easily done in a spreadsheet.
- **Have a visualization** (chart or map) of some kind
- Don't leave any sensitive information in the notebook: API keys, personally-identifiable information (PII), etc.

If you answer the first question easily, that's fine; dig into / build off of it. Go deep, not broad.

## No homework/resubmissions will be accepted after Thursday 5/12 at 6:45pm ET

In other words, homework 6 cannot be late.

## Lecture 6